In [1]:
import os

from dotenv import load_dotenv, find_dotenv

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain.llms import Ollama


## Simple Chain

In [2]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = Ollama(model="llama3.2:1b", temperature=0)
output_parser = StrOutputParser()

In [3]:
chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [5]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [6]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="llama3.2:1b" )
)
retriever = vectorstore.as_retriever()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [7]:
retriever.get_relevant_documents("where did harrison work?")

/tmp/ipykernel_27173/3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [8]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain.schema.runnable import RunnableMap

In [11]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [12]:
chain.invoke({"question": "where did harrison work?"})

'Based on the provided context, it can be inferred that Harrison worked at Kensho.'

In [13]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind


In [14]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = Ollama(model="llama3.2:1b",temperature=0).bind(functions=functions)

In [16]:
runnable = prompt | model

In [17]:
runnable.invoke({"input": "what is the weather in sf"})

"As of my knowledge cutoff in March 2023, San Francisco's weather can be quite unpredictable and varied depending on the time of year. However, I can give you a general idea of what to expect.\n\nIn the winter months (December to February), San Francisco typically experiences mild temperatures, with average highs around 58°F (14°C) and lows around 45°F (7°C). It's not uncommon for there to be foggy mornings, especially in January and February. The city can also experience light rain showers throughout the month.\n\nIn the spring (March to May), San Francisco's weather is generally mild and pleasant, with average highs ranging from 64°F (18°C) to 73°F (23°C). The temperatures are usually warm enough for outdoor activities like hiking or biking, but it's still a good idea to pack layers for cooler mornings and evenings.\n\nSummer (June to August) in San Francisco is typically hot and dry, with average highs reaching the mid-80s to low 90s (29°C to 32°C). The temperatures can soar during 

In [18]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [19]:
model = model.bind(functions=functions)

In [20]:
runnable = prompt | model

In [21]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

"I'm happy to help, but I don't have any information about a specific game or event involving the New England Patriots. However, I can tell you that the Patriots are a professional American football team that competes in the National Football League (NFL). If you're looking for information about their performance yesterday, I recommend checking out reputable sports news sources or the official NFL website for updates and analysis."

## Fallbacks


In [22]:
import json
simple_model = Ollama(model="llama3.2:1b", temperature=0)

simple_chain = simple_model | json.loads

In [23]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line, response with only json style"

In [24]:
resp=simple_model.invoke(challenge)
resp

'```\n[\n  {\n    "title": "The Road Not Taken",\n    "author": "Robert Frost",\n    "firstLine": "Two roads diverged in a yellow wood"\n  },\n  {\n    "title": "The Love Song of J. Alfred Prufrock",\n    "author": "T.S. Eliot",\n    "firstLine": "In the room the women come and go"\n  },\n  {\n    "title": "Do Not Go Gentle into That Good Night",\n    "author": "Dylan Thomas",\n    "firstLine": "Rage, rage against the dying of the light"\n  }\n]\n```'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [25]:
simple_chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [26]:
from langchain.chains import TransformChain, LLMChain

# Função para converter resposta em JSON
def resp_to_json(resp):
    limpa_string = resp.replace('```json', '').replace('```', '').strip()
    try:
        dados = json.loads(limpa_string)
        return dados
    except json.JSONDecodeError as e:
        print(e)
        return None

# Usando TransformChain para aplicar o resp_to_json
json_transform = TransformChain(
    input_variables=["text"],
    output_variables=["json"],
    transform=lambda inputs: {"json": resp_to_json(inputs["text"])}
)

In [27]:
model = Ollama(model="llama3.3:latest", temperature=0)

chain = model | StrOutputParser() | json_transform
chain.invoke(challenge)

{'text': '```\n{\n  "poems": [\n    {\n      "title": "The Road Not Taken",\n      "author": "Robert Frost",\n      "firstLine": "Two roads diverged in a yellow wood"\n    },\n    {\n      "title": "The Love Song of J. Alfred Prufrock",\n      "author": "T.S. Eliot",\n      "firstLine": "Let us go then, you and I"\n    },\n    {\n      "title": "Do Not Go Gentle into That Good Night",\n      "author": "Dylan Thomas",\n      "firstLine": "Do not go gentle into that good night"\n    }\n  ]\n}\n```',
 'json': {'poems': [{'title': 'The Road Not Taken',
    'author': 'Robert Frost',
    'firstLine': 'Two roads diverged in a yellow wood'},
   {'title': 'The Love Song of J. Alfred Prufrock',
    'author': 'T.S. Eliot',
    'firstLine': 'Let us go then, you and I'},
   {'title': 'Do Not Go Gentle into That Good Night',
    'author': 'Dylan Thomas',
    'firstLine': 'Do not go gentle into that good night'}]}}

In [28]:
final_chain = simple_chain.with_fallbacks([chain])
final_chain.invoke(challenge)

{'text': '```\n{\n  "poems": [\n    {\n      "title": "The Road Not Taken",\n      "author": "Robert Frost",\n      "firstLine": "Two roads diverged in a yellow wood"\n    },\n    {\n      "title": "The Love Song of J. Alfred Prufrock",\n      "author": "T.S. Eliot",\n      "firstLine": "Let us go then, you and I"\n    },\n    {\n      "title": "Do Not Go Gentle into That Good Night",\n      "author": "Dylan Thomas",\n      "firstLine": "Do not go gentle into that good night"\n    }\n  ]\n}\n```',
 'json': {'poems': [{'title': 'The Road Not Taken',
    'author': 'Robert Frost',
    'firstLine': 'Two roads diverged in a yellow wood'},
   {'title': 'The Love Song of J. Alfred Prufrock',
    'author': 'T.S. Eliot',
    'firstLine': 'Let us go then, you and I'},
   {'title': 'Do Not Go Gentle into That Good Night',
    'author': 'Dylan Thomas',
    'firstLine': 'Do not go gentle into that good night'}]}}

## Interface

In [29]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [30]:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!'

In [31]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!',
 'Why did the frog go to the doctor?\n\nBecause it wasn\'t feeling "toad-ally" well!']

In [32]:
response = await chain.ainvoke({"topic": "bears"})


In [33]:
for t in chain.stream({"topic": "bears"}):
    print(t)

Why
 did
 the
 bear
 go
 to
 the
 doctor
?


Because
 it
 had
 a
 gr
izzly
 cough
!



## Pydantic  function definition

In [34]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
from typing import List
from pydantic import BaseModel, Field

In [37]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")
    
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/tmp/ipykernel_27173/3675743362.py:6: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [38]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [40]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [41]:
model = Ollama(model="llama3.2:1b", temperature=0)


In [42]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

'I\'m not aware of your current location. However, I can suggest some options to help you find out the weather in San Francisco.\n\nYou can:\n\n1. Check online weather websites such as AccuWeather, Weather.com, or the National Weather Service (NWS) for the most up-to-date forecast.\n2. Use a search engine like Google to look up "weather in San Francisco" or "San Francisco temperature today".\n3. Download a weather app on your smartphone, such as Dark Sky or Weather Underground, which can provide you with real-time weather information.\n\nPlease let me know if there\'s anything else I can help you with.'

In [43]:
model_with_function = model.bind(functions=[weather_function])

In [44]:
model_with_function.invoke("what is the weather in sf?")

"San Francisco, California has a mild climate year-round. Here's an overview of the typical weather conditions:\n\n**Seasonal Weather Patterns:**\n\n* **Winter (December to February):** Mild and cool, with average highs around 58°F (14°C) and lows around 45°F (7°C). Expect occasional foggy mornings.\n* **Spring (March to May):** Cool and breezy, with average highs around 63°F (17°C) and lows around 50°F (10°C). Spring is a great time to visit San Francisco, with mild weather and fewer tourists.\n* **Summer (June to August):** Warm and sunny, with average highs around 73°F (23°C) and lows around 58°F (14°C). Summer is peak tourist season in San Francisco, but it can get crowded.\n* **Autumn (September to November):** Mild and pleasant, with average highs around 68°F (20°C) and lows around 52°F (11°C). Autumn is a great time to visit San Francisco, with comfortable weather and fewer tourists.\n\n**Regional Weather Variations:**\n\n* **Coastal Areas:** The coastal areas of San Francisco, 

## Forcing it to use a function

We can force the model to use a function

In [46]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [47]:
model_with_forced_function.invoke("what is the weather in sf?")

"San Francisco, California has a mild climate year-round. Here's an overview of the typical weather conditions:\n\n**Seasonal Weather Patterns:**\n\n* **Winter (December to February):** Mild and cool, with average highs around 58°F (14°C) and lows around 45°F (7°C). Expect occasional foggy mornings.\n* **Spring (March to May):** Cool and breezy, with average highs around 65°F (18°C) and lows around 50°F (10°C). Spring is a great time to visit San Francisco, with mild weather and fewer tourists.\n* **Summer (June to August):** Warm and sunny, with average highs around 75°F (24°C) and lows around 55°F (13°C). Summer is peak tourist season in San Francisco, but it can get crowded.\n* **Autumn (September to November):** Mild and pleasant, with average highs around 65°F (18°C) and lows around 50°F (10°C). Autumn is a great time to visit San Francisco, with comfortable weather and fewer tourists.\n\n**Regional Weather Variations:**\n\n* **Coastal Areas:** The coastal areas of San Francisco, 

In [48]:
model_with_forced_function.invoke("hi!")

'Hello! How can I help you today?'

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [49]:
from langchain.prompts import ChatPromptTemplate

In [50]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [51]:
chain = prompt | model_with_function

In [66]:
chain.invoke({"input": "what is the weather in sf?"})

"The weather in San Francisco, California. As of my knowledge cutoff in March 2023, the average high temperature in San Francisco during the summer months (June to August) is around 68°F (20°C), while the average low temperature is around 55°F (13°C).\n\nIn the winter months (December to February), the average high temperature is around 58°F (14°C), while the average low temperature is around 45°F (7°C). However, it's not uncommon for the weather in San Francisco to be quite unpredictable and change quickly.\n\nIt's also worth noting that San Francisco is known for its foggy climate, especially during the winter months. The city can experience fog rolling in from the Pacific Ocean as early as 5 am, and it can last until around 10 pm or midnight.\n\nIf you're planning a trip to San Francisco, I recommend checking the weather forecast before your trip to get a better idea of what to expect. You can check websites like AccuWeather or Weather.com for up-to-date forecasts and conditions."

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [67]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [68]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [69]:
model_with_functions = model.bind(functions=functions)

In [70]:
model_with_functions.invoke("what is the weather in sf?")

"San Francisco, California has a mild climate year-round. Here's an overview of the typical weather conditions:\n\n**Seasonal Weather Patterns:**\n\n* **Winter (December to February):** Mild and cool, with average highs around 58°F (14°C) and lows around 45°F (7°C). Expect occasional foggy mornings.\n* **Spring (March to May):** Cool and breezy, with average highs around 65°F (18°C) and lows around 50°F (10°C). Spring is a great time to visit San Francisco, with mild weather and fewer tourists.\n* **Summer (June to August):** Warm and sunny, with average highs around 75°F (24°C) and lows around 55°F (13°C). Summer is peak tourist season in San Francisco.\n* **Autumn (September to November):** Mild and pleasant, with average highs around 65°F (18°C) and lows around 50°F (10°C). Autumn is a great time to visit San Francisco, with comfortable weather and fewer crowds.\n\n**Regional Weather Variations:**\n\n* **Coastal Areas:** The coastal areas of San Francisco, such as Fisherman's Wharf 

In [71]:
model_with_functions.invoke("what are three songs by taylor swift?")

'Here are three songs by Taylor Swift:\n\n1. "Shake It Off"\n2. "Blank Space"\n3. "Love Story"'

In [72]:
model_with_functions.invoke("hi!")

'Hello! How can I help you today?'

In [73]:
model_with_functions

RunnableBinding(bound=Ollama(model='llama3.2:1b', temperature=0.0), kwargs={'functions': [{'name': 'WeatherSearch', 'description': 'Call this with an airport code to get the weather at that airport', 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for', 'type': 'string'}}, 'required': ['airport_code'], 'type': 'object'}}, {'name': 'ArtistSearch', 'description': 'Call this to get the names of songs by a particular artist', 'parameters': {'properties': {'artist_name': {'description': 'name of artist to look up', 'type': 'string'}, 'n': {'description': 'number of results', 'type': 'integer'}}, 'required': ['artist_name', 'n'], 'type': 'object'}}]})